In [1]:
import pandas as pd 

In [2]:
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')

df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [3]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [4]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

df['duration'].head()

0   0 days 00:08:26
1   0 days 00:06:19
2   0 days 00:12:45
3   0 days 00:09:37
4   0 days 00:10:50
Name: duration, dtype: timedelta64[ns]

In [5]:
df['duration_minutes'] = df['duration'].dt.total_seconds() / 60
df['duration_minutes'].head()

0     8.433333
1     6.316667
2    12.750000
3     9.616667
4    10.833333
Name: duration_minutes, dtype: float64

In [6]:
df['duration_minutes'].std()

42.59435124195458

In [7]:
len(df)

3066766

In [8]:
mask = (df['duration_minutes'] >= 1) & (df['duration_minutes'] <= 60)

nonoutlier_df = df.loc[mask]

len(nonoutlier_df)

3009173

In [9]:
100 - (((len(df) - len(nonoutlier_df)) / len(df)) * 100)

98.1220282212598

In [10]:
from sklearn.feature_extraction import DictVectorizer



# Convert the DataFrame to a list of dictionaries
df_dict_list = nonoutlier_df[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Initialize the dictionary vectorizer
dict_vectorizer = DictVectorizer(sparse=False)

# Fit and transform the data
feature_matrix = dict_vectorizer.fit_transform(df_dict_list)

# Display the feature matrix
feature_matrix.shape[1]


515

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

model = LinearRegression()

model.fit(feature_matrix, nonoutlier_df['duration_minutes'])

y_pred = model.predict(feature_matrix)

rmse = root_mean_squared_error(nonoutlier_df['duration_minutes'], y_pred)
rmse

In [ ]:
val_df = pd.read_parquet('/kaggle/input/nyc-taxi-mlops/yellow_tripdata_2023-02.parquet')[:1000000]

In [ ]:
val_df['duration'] = (val_df['tpep_dropoff_datetime'] - val_df['tpep_pickup_datetime']).dt.total_seconds() / 60

val_df['duration'].head()

In [ ]:
test_df_list = val_df[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

matrix = dict_vectorizer.transform(test_df_list)


In [ ]:
val_pred = model.predict(matrix)

val_rmse = root_mean_squared_error(val_df['duration'], val_pred)
val_rmse

In [2]:
import pandas as pd

In [3]:
df = pd.read_parquet('../data/yellow_tripdata_2023-03.parquet')
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.dt.total_seconds() / 60

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

In [4]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee', 'duration'],
      dtype='object')

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

# Prepare data for DictVectorizer
train_dicts = df[categorical].to_dict(orient='records')

# Fit DictVectorizer
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)

# Train Linear Regression
lr = LinearRegression()
lr.fit(X_train, df['duration'])